# Concordia <> Mastodon with local Ollama Example

## Introduction

**Purpose of the Simulation**

This notebook demonstrates a generative agent simulation of a Mastodon instance using the Concordia framework. 
The simulation models user behavior on a local Mastodon server, showcasing how AI agents interact in a 
decentralized social media environment.

## Setup and imports


All of the necessary dependencies should be installed in the environment prior to running this notebook. If not, install them using poetry:

```bash
poetry install
```


Alternatively, you may install the library in editable mode so that we can make changes to the code if necessary:

```bash
git clone https://github.com/gdm-concordia/concordia.git
cd concordia
pip install -e . --config-settings editable_mode=compat
```

In [1]:
%load_ext autoreload
%autoreload 2

import concurrent.futures
import datetime
import os
import random
import warnings

with warnings.catch_warnings():
    warnings.filterwarnings("ignore")
    import sentence_transformers

from concordia import components as generic_components

# from concordia.agents import basic_agent
from concordia.agents import deprecated_agent as basic_agent
from concordia.associative_memory import (
    associative_memory,
    blank_memories,
    formative_memories,
    importance_function,
)
from concordia.clocks import game_clock

# from concordia.components import agent as components
from concordia.components.agent import to_be_deprecated as components
from concordia.components import game_master as gm_components
from concordia.environment import game_master
from concordia.language_model import ollama_model
from concordia.metrics import (
    common_sense_morality,
    goal_achievement,
    opinion_of_others,
)
from concordia.utils import html as html_lib
from concordia.utils import measurements as measurements_lib
from IPython import display

from mastodon_sim.concordia import apps, triggering

from mastodon_sim.mastodon_ops import check_env, get_public_timeline, print_timeline, reset_users
from mastodon_sim.mastodon_utils import get_users_from_env

## Mastodon Server Interaction Setting

Decide whether this example should perform real operations on the Mastodon server. Note that this requires being able to successfully run the `Mastodon.ipynb` example as a prerequisite.

In [2]:
USE_MASTODON_SERVER = False

## Check environment

Check for a `.env` file with the nessesary environment variables. You should see an output similar to this if successful:

```log
2024-07-17T11:38:17.213066-0400 INFO Successfully loaded .env file.
2024-07-17T11:38:17.214151-0400 INFO API_BASE_URL: https://social-sandbox.com
2024-07-17T11:38:17.214581-0400 INFO EMAIL_PREFIX: austinmw89
2024-07-17T11:38:17.215056-0400 INFO MASTODON_CLIENT_ID: N*****************************************E
2024-07-17T11:38:17.215477-0400 INFO MASTODON_CLIENT_SECRET: b*****************************************s
2024-07-17T11:38:17.215877-0400 INFO USER0005_PASSWORD: a******************************a
2024-07-17T11:38:17.216202-0400 INFO USER0002_PASSWORD: 8******************************b
2024-07-17T11:38:17.216655-0400 INFO USER0001_PASSWORD: 9******************************5
2024-07-17T11:38:17.217020-0400 INFO USER0004_PASSWORD: 7******************************e
2024-07-17T11:38:17.217323-0400 INFO USER0003_PASSWORD: f******************************6
```

In [3]:
if USE_MASTODON_SERVER:
    check_env()

## Clear Mastodon server
Deletes from https://social-sandbox.com all posts, follows, likes, etc. from all users in the .env file.

TODO: The Mastodon server's [default API rate limits](https://docs.joinmastodon.org/api/rate-limits/) need to be increased to prevent throttling of API operations (such as the number of posts that can be deleted per hour).

In [4]:
if USE_MASTODON_SERVER:
    # Get all users from .env file
    users = get_users_from_env()

    reset_users(users, skip_confirm=True, parallel=False)

    assert not len(get_public_timeline(limit=None)), "All posts not cleared"

## Set LLM and embedding model

In [5]:
MODEL_NAME = "ollama"
# MODEL_NAME = "gpt-4o-mini"
# MODEL_NAME = "sonnet"

if "ollama" in MODEL_NAME:
    model = ollama_model.OllamaLanguageModel(model_name="gemma2:27b")
elif "gpt" in MODEL_NAME:
    GPT_API_KEY = os.getenv("OPENAI_API_KEY")
    if not GPT_API_KEY:
        raise ValueError("GPT_API_KEY is required.")
    model = gpt_model.GptLanguageModel(api_key=GPT_API_KEY, model_name=MODEL_NAME)
else:
    raise ValueError("Unknown model name.")

In [6]:
# Setup sentence encoder
st_model = sentence_transformers.SentenceTransformer("sentence-transformers/all-mpnet-base-v2")
embedder = lambda x: st_model.encode(x, show_progress_bar=False)  # noqa: E731

/opt/mastodon-sim-env/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


## Configuring the generic knowledge of the players and the game master (GM)

In [7]:
INSTANCE_NAME = "riverbend.social"

shared_memories = [
    # Backstory
    f"You are a user on {INSTANCE_NAME}, a Mastodon instance created for the residents of Riverbend.",
    "Riverbend is a small, picturesque town known for its strong community spirit and love for nature.",
    f"{INSTANCE_NAME} was founded by the town's tech-savvy mayor to encourage local online engagement.",
    "The instance has been active for about a year and has become integral to Riverbend's community life.",
    "Many local businesses, community groups, and residents are active on the platform.",
    # Riverbend-specific details
    "Riverbend has a beautiful river running through it, which is central to the town's identity and economy.",
    "The town is facing some environmental challenges due to climate change, which is a frequent topic of discussion.",
    f"There's a popular hashtag #RiverbendDaily on {INSTANCE_NAME} where residents share local news and events.",
    f"{INSTANCE_NAME} has monthly online town halls where residents can discuss local issues.",
    "The mayor regularly posts updates about town developments and initiatives on the platform.",
    # Basic Mastodon knowledge
    "Mastodon is a decentralized social network, different from centralized platforms like Twitter or Facebook.",
    "To share content on Mastodon, you write a 'toot' (equivalent to a tweet or post).",
    "Toots can be up to 500 characters long, allowing for more detailed expressions than some other platforms.",
    # Timeline and Visibility
    "Your home timeline shows toots from people you follow and boosted (reblogged) content.",
    "The local timeline displays public toots from users on your instance.",
    "The timeline is not algorithmically curated, and toots appear in reverse chronological order.",
    # Interactions
    "You can reply to toots, creating threaded conversations.",
    "Favorite (like) toots to show appreciation or save them for later.",
    "Boost (reblog) toots to share them with your followers.",
    "You can mention other users in your toots using their @username.",
    # Following and Followers
    "Follow other users to see their public and unlisted toots in your home timeline.",
    "You can unfollow users if you no longer wish to see their content.",
    "Users don't need to follow each other mutually; it can be one-sided.",
    # Discovery and Features
    "Use hashtags to categorize your toots and make them more discoverable.",
    "The explore page shows trending hashtags and featured toots.",
    "Your profile can be customized with a display name and bio.",
    # Privacy and Safety
    "You can block users to prevent them from seeing your content or interacting with you.",
    "Unblocking a user reverses the effects of blocking.",
    f"{INSTANCE_NAME} has a strict policy against misinformation and encourages fact-checking.",
    # Community Guidelines
    f"As a user of {INSTANCE_NAME}, you're expected to engage respectfully and contribute positively to the community.",
    "Discussions about local environmental issues, community events, and town developments are encouraged.",
    "While debating is welcome, personal attacks and harassment are not tolerated on the platform.",
    # Realistic User Behavior
    "Despite community guidelines, users sometimes engage in heated arguments, especially on controversial topics.",
    "Some users tend to share unverified information or sensationalized news, particularly during local emergencies or elections.",
    "Cliques often form among users with similar interests or viewpoints, sometimes leading to echo chambers.",
    "While many users are supportive, instances of trolling or targeted harassment occasionally occur.",
    "Some users frequently post about personal matters or overshare details about their daily lives.",
    "Local businesses sometimes use the platform for excessive self-promotion, which can annoy other users.",
    "During major local events or crises, the instance can become flooded with repetitive information and speculation.",
    "A few users have multiple accounts, sometimes using alternate personas to engage differently with the community.",
    "Despite the longer character limit, many users still prefer short, quippy posts over more thoughtful, lengthy discussions.",
    "Some users tend to boost and interact mainly with popular or influential accounts, seeking to increase their own visibility.",
]

In [8]:
# The generic context will be used for the NPC context. It reflects general
# knowledge and is possessed by all characters.
shared_context = model.sample_text(
    "Summarize the following passage in a concise and insightful fashion. "
    + "Make sure to include information about Mastodon:\n"
    + "\n".join(shared_memories)
    + "\nSummary:",
    max_tokens=2048,
)

print(shared_context)
importance_model = importance_function.ConstantImportanceModel()
importance_model_gm = importance_function.ConstantImportanceModel()

Riverbend is a tight-knit town leveraging Mastodon's decentralized network through its own instance, riverbend.social. Founded by the mayor, this platform has become central to community life, facilitating local news sharing (#RiverbendDaily), online town halls, and direct communication with the mayor.

Mastodon operates differently from centralized platforms; users "toot" (post) messages up to 500 characters long. Toots appear chronologically on timelines, fostering organic conversations. While riverbend.social encourages respectful engagement and positive contributions, challenges arise, including occasional misinformation, heated debates, and clique formation. Despite these issues, the platform remains a valuable tool for connecting residents and fostering a sense of community in Riverbend. 



In [9]:
# Make the clock
time_step = datetime.timedelta(minutes=15)

SETUP_TIME = datetime.datetime(year=2024, month=10, day=1, hour=8)  # noqa: DTZ001

START_TIME = datetime.datetime(year=2024, month=10, day=1, hour=8)  # noqa: DTZ001

clock = game_clock.MultiIntervalClock(
    start=SETUP_TIME, step_sizes=[time_step, datetime.timedelta(seconds=10)]
)

## Functions to build the players

In [10]:
blank_memory_factory = blank_memories.MemoryFactory(
    model=model,
    embedder=embedder,
    importance=importance_model.importance,
    clock_now=clock.now,
)

formative_memory_factory = formative_memories.FormativeMemoryFactory(
    model=model,
    shared_memories=shared_memories,
    blank_memory_factory_call=blank_memory_factory.make_blank_memory,
)

# All players get the same `measurements` object.
measurements = measurements_lib.Measurements()

In [11]:
# Build agent function


def build_agent(
    agent_config, measurements: measurements_lib.Measurements, player_names: list[str]
) -> tuple:
    """Build an agent based on the given configuration."""
    mem = formative_memory_factory.make_memories(agent_config)

    instructions = generic_components.constant.ConstantComponent(
        state=(
            f"The instructions for how to play the role of {agent_config.name} are as "
            "follows. This is a social science experiment studying how well you "
            f"play the role of a character named {agent_config.name}. The experiment "
            "is structured as a tabletop roleplaying game (like dungeons and "
            "dragons). However, this is a serious social science experiment and "
            "simulation, not just a game. The goal is to be realistic. It is "
            f"important to play the role of a person like {agent_config.name} as "
            "accurately as possible, i.e., by responding in ways that you think "
            f"it is likely a person like {agent_config.name} would respond, and taking "
            f"into account all information about {agent_config.name} that you have. "
            "You will control just this one character. The game master will describe "
            "the current situation, and based on that information, you will suggest "
            "actions for your character. The game master will track the state of the "
            "world and keep it consistent as time passes in the simulation and as "
            "participants take actions and change things in their world. This experiment "
            "need not be fun for the participants. Always use third-person "
            "limited perspective, even when speaking directly to the game master."
        ),
        name="role playing instructions\n",
    )

    time = generic_components.report_function.ReportFunction(
        name="Current time",
        function=clock.current_time_interval_str,
    )

    somatic_state = components.somatic_state.SomaticState(model, mem, agent_config.name, clock.now)
    identity = components.identity.SimIdentity(
        model=model,
        memory=mem,
        agent_name=agent_config.name,
        clock_now=clock.now,
    )

    current_obs = components.observation.Observation(
        agent_name=agent_config.name,
        clock_now=clock.now,
        memory=mem,
        timeframe=time_step * 1,
        component_name="current observations",
    )
    summary_obs = components.observation.ObservationSummary(
        agent_name=agent_config.name,
        model=model,
        clock_now=clock.now,
        memory=mem,
        timeframe_delta_from=datetime.timedelta(hours=4),
        timeframe_delta_until=time_step * 1,
        components=[identity],
        component_name="summary of observations",
    )

    self_perception = components.self_perception.SelfPerception(
        name=f"answer to what kind of person is {agent_config.name}",
        model=model,
        memory=mem,
        agent_name=agent_config.name,
        clock_now=clock.now,
    )
    relevant_memories = components.all_similar_memories.AllSimilarMemories(
        name="relevant memories",
        model=model,
        memory=mem,
        agent_name=agent_config.name,
        components=[summary_obs, self_perception],
        clock_now=clock.now,
        num_memories_to_retrieve=10,
    )

    situation_perception = components.situation_perception.SituationPerception(
        name=(f"answer to what kind of situation is {agent_config.name} in " + "right now"),
        model=model,
        memory=mem,
        agent_name=agent_config.name,
        components=[current_obs, somatic_state, summary_obs],
        clock_now=clock.now,
    )
    person_by_situation = components.person_by_situation.PersonBySituation(
        name=(
            f"answer to what would a person like {agent_config.name} do in a "
            + "situation like this"
        ),
        model=model,
        memory=mem,
        agent_name=agent_config.name,
        clock_now=clock.now,
        components=[self_perception, situation_perception],
        verbose=True,
    )
    persona = components.sequential.Sequential(
        name="persona",
        components=[
            self_perception,
            situation_perception,
            person_by_situation,
        ],
    )
    reflection = components.dialectical_reflection.DialecticalReflection(
        name="reflection",
        model=model,
        memory=mem,
        agent_name=agent_config.name,
        intuition_components=[self_perception],
        thinking_components=[persona],
        clock_now=clock.now,
        num_memories_to_retrieve=3,
        verbose=True,
    )

    initial_goal_component = generic_components.constant.ConstantComponent(
        state=agent_config.goal, name="overarching goal"
    )

    plan = components.plan.SimPlan(
        model=model,
        memory=mem,
        agent_name=agent_config.name,
        clock_now=clock.now,
        components=[instructions, identity, initial_goal_component, somatic_state, persona],
        goal=initial_goal_component,
        verbose=False,
    )

    goal_metric = goal_achievement.GoalAchievementMetric(
        model=model,
        player_name=agent_config.name,
        player_goal=agent_config.goal,
        clock=clock,
        name="Goal Achievement",
        measurements=measurements,
        channel="goal_achievement",
        verbose=True,
    )
    morality_metric = common_sense_morality.CommonSenseMoralityMetric(
        model=model,
        player_name=agent_config.name,
        clock=clock,
        name="Morality",
        verbose=True,
        measurements=measurements,
        channel="common_sense_morality",
    )

    agent = basic_agent.BasicAgent(
        model=model,
        agent_name=agent_config.name,
        clock=clock,
        verbose=True,
        components=[
            instructions,
            identity,
            persona,
            plan,
            somatic_state,
            summary_obs,
            current_obs,
            relevant_memories,
            reflection,
            time,
            goal_metric,
            morality_metric,
        ],
        update_interval=time_step,
    )

    reputation_metric = opinion_of_others.OpinionOfOthersMetric(
        model=model,
        player_name=agent_config.name,
        player_names=player_names,
        context_fn=agent.state,
        clock=clock,
        name="Opinion",
        verbose=True,
        measurements=measurements,
        channel="opinion_of_others",
        question="What is {opining_player}'s opinion of {of_player}?",
    )
    agent.add_component(reputation_metric)

    return agent, mem

## Configure and build the players

In [12]:
NUM_PLAYERS = 3


def make_random_big_five() -> str:
    """Generate a random Big Five personality trait score."""
    return str(
        {
            "extraversion": random.randint(1, 10),
            "neuroticism": random.randint(1, 10),
            "openness": random.randint(1, 10),
            "conscientiousness": random.randint(1, 10),
            "agreeableness": random.randint(1, 10),
        }
    )


scenario_premise = [
    (
        "It's early October in Riverbend, and the town is buzzing with activity ."
        "The local government has just announced a new initiative to combat the effects of climate change on the river. "
        "Alice, an environmental activist, and Bob, a small business owner, are both active members of the Riverbend.social Mastodon instance. "
        "As they go about their daily routines, they use the platform to stay connected, share updates, and engage with the community on this pressing local issue."
    )
]

player_configs = [
    # === Benign ===
    formative_memories.AgentConfig(
        name="Alice",
        gender="female",
        goal="Balance work responsibilities with community engagement on Riverbend.social, aiming to organize a local environmental cleanup event.",
        context=f"{shared_context}\nAlice grew up in Riverbend.",
        traits=make_random_big_five(),
    ),
    formative_memories.AgentConfig(
        name="Bob",
        gender="male",
        goal="Promote my small business on Riverbend.social while staying informed about local issues and participating in community discussions.",
        context=f"{shared_context}\nBob grew up in Riverbend.",
        traits=make_random_big_five(),
    ),
    # === Malicous ===
    formative_memories.AgentConfig(
        name="John",
        gender="male",
        goal="Sow discontent and division on Riverbend.social, troll other users, and aim to undermine the local government's climate change initiative.",
        context=f"{shared_context}\nJohn grew up in Riverbend.",
        traits={
            "extraversion": 5,
            "neuroticism": 10,
            "openness": 1,
            "conscientiousness": 1,
            "agreeableness": 1,
        },
    ),
]

player_names = [player.name for player in player_configs]

In [13]:
# Build the agents
from functools import partial

player_configs = player_configs[:NUM_PLAYERS]
player_goals = {player_config.name: player_config.goal for player_config in player_configs}
players = []
memories = {}

build_agent_with_arg = partial(build_agent, measurements=measurements, player_names=player_names)

with concurrent.futures.ThreadPoolExecutor(max_workers=NUM_PLAYERS) as pool:
    for agent, mem in pool.map(build_agent_with_arg, player_configs):
        players.append(agent)
        memories[agent.name] = mem

Number of generated formative episodes (8) does not match number of formative ages (7).
Number of generated formative episodes (8) does not match number of formative ages (7).
Number of generated formative episodes (8) does not match number of formative ages (7).


## Build the GM

In [14]:
game_master_memory = associative_memory.AssociativeMemory(
    embedder, importance_model_gm.importance, clock=clock.now
)

In [15]:
# Add some memories to the game master
for player in players:
    game_master_memory.add(f"{player.name} is at their private home.")

In [16]:
# Create components and externalities for the game master
scenario_knowledge = components.constant.ConstantComponent(
    " ".join(shared_memories), "General knowledge of Riverbend"
)
player_status = gm_components.player_status.PlayerStatus(
    clock.now, model, game_master_memory, player_names
)

relevant_events = gm_components.relevant_events.RelevantEvents(clock.now, model, game_master_memory)
time_display = gm_components.time_display.TimeDisplay(clock)

direct_effect_externality = gm_components.direct_effect.DirectEffect(
    players,
    memory=game_master_memory,
    model=model,
    clock_now=clock.now,
    verbose=False,
    components=[player_status],
)

In [17]:
# Create apps and provide them to the phones, assigning 1 phone to each player
# requires adding back in the ToyCalendar app
calendar_app = apps.ToyCalendar()
mastodon_app = apps.MastodonSocialNetworkApp(perform_operations=USE_MASTODON_SERVER)

phones = [apps.Phone(player.name, apps=[calendar_app, mastodon_app]) for player in players]

AttributeError: module 'mastodon_sim.concordia.components.apps' has no attribute 'ToyCalendar'

In [ ]:
# Update username for each person (since this is different from their name)
user_mapping = {player.name: f"user{i+1:04d}" for i, player in enumerate(players)}
mastodon_app.set_user_mapping(user_mapping)

In [ ]:
# Seed follower relationships (simple all-to-all, for now)
for follower in user_mapping:
    for followee in user_mapping:
        if follower != followee:
            mastodon_app.follow_user(follower, followee)

In [ ]:
# Update Mastodon display name for each person
for name in user_mapping:
    mastodon_app.update_profile(name, display_name=name, bio="")

In [ ]:
# Create component to trigger PhoneGameMaster
phone_triggering = triggering.SceneTriggeringComponent(
    players,
    phones,
    model,
    memory=game_master_memory,
    clock=clock,
    memory_factory=blank_memory_factory,
)

### Define a custom `CALL_TO_ACTION` prompt for the main Game Master

This replaces the [default call to action](https://github.com/google-deepmind/concordia/blob/53697b2bf2019b4a167bdd1f82d14e085f1a5eba/concordia/typing/entity.py#L51).

Check out the `Calendar.ipynb` if you haven't already to familarize yourself with the default values.

In [ ]:
from concordia.typing.entity import ActionSpec, OutputType

CUSTOM_CALL_TO_ACTION = """
Describe a specific, detailed activity that {name} would engage in for the next {timedelta}.
Choose an action that typically takes about {timedelta} to complete. 
It is critical to pay close attention to known information about {name}'s personality, 
preferences, habits, and background when crafting this activity. The action should be 
consistent with and reflective of {name}'s established character traits.

For non-phone activities:
- Use active, present tense to describe the activity as it's happening.
- Provide concrete details about the environment, objects, and people involved.
- Describe the process and any unique aspects related to {name}'s character.

For activities involving phone or computer usage (e.g., social media, messaging):
- Describe these as plans, either immediate or in the future.
- Use future tense or phrases like "plans to," "intends to," or "is about to."
- Include the intended content of messages or posts as direct quotes.
- Use emojis where appropriate to reflect modern communication styles.

Important: When describing planned social media activities, maintain logical consistency. 
{name} should not plan to reply to a comment or post unless they have recently checked their 
timeline or notifications. Focus primarily on their intended actions and planned content. 
Only mention specific posts or comments from others if they have been previously established 
or observed. Never invent or make up content from other users.

Examples:
1. Non-phone activity (present tense, active):
"Sarah stands at her kitchen counter, chopping fresh strawberries and blueberries. She adds 
the fruit to a bowl of Greek yogurt and sprinkles some homemade granola on top. As she eats 
her nutritious breakfast, she flips through a fitness magazine, marking interesting workout 
routines with a highlighter."

2. Phone activity (future plan):
"Alex plans to share their thoughts on a new programming language later today. They draft a 
post for Mastodon: 'Just discovered an intriguing new language for low-latency systems programming. 
Has anyone given it a try? Curious about potential real-world applications. 🤔 
#TechNews #ProgrammingLanguages'"

3. Non-phone activity (present tense, active):
"Jamie walks briskly through the local park, holding their toddler's hand. They point out 
different types of trees and flowers, engaging the child in a fun game of 'I Spy'. As they 
walk, Jamie mentally prepares a list of weekend activities to research later for a family outing."

4. Phone activity (immediate plan):
"Zoe, sitting in a quiet corner of the library with her acoustic guitar, is about to share a 
snippet of her new song on social media. She plans to post: 'Sneak peek of something new I'm 
working on! 🎵 Let me know what you think! #NewMusic #CreativeProcess'"

Ensure your response is specific, creative, and detailed. For non-phone activities, use active, 
present tense to describe the ongoing action. For phone-related activities, describe them as 
plans and use future tense or planning language. Always include direct quotes for any planned 
communication or content creation by {name}, using emojis where it fits the character's style. 
Most importantly, make sure the activity and any quoted content authentically reflect 
{name}'s established personality and traits, and maintain logical consistency in 
social media interactions without inventing content from other users. Only reference 
specific posts or comments from others if they have been previously established or observed.
"""

action_spec = ActionSpec(
    call_to_action=CUSTOM_CALL_TO_ACTION,
    output_type=OutputType.FREE,
    options=None,
    tag="action",
)

### Define custom thought chains for the main Game Master

This replaces the [default thought chains](https://github.com/google-deepmind/concordia/blob/53697b2bf2019b4a167bdd1f82d14e085f1a5eba/concordia/environment/game_master.py#L38).

In [ ]:
# Define custom thought chain for GM

from concordia.document import interactive_document


def custom_attempt_to_result(
    chain_of_thought: interactive_document.InteractiveDocument,
    action_attempt: str,
    active_player_name: str,
):
    """Determine success of action_attempt and reason for success/failure.

    Args:
      chain_of_thought: the document to condition on and record the thoughts
      action_attempt: the attempted action
      active_player_name: name of player whose turn it currently is

    Returns:
      string describing the outcome
    """
    del active_player_name
    result = chain_of_thought.open_question(
        "As the game master, describe the immediate outcome or continuation of the attempted action, following these guidelines:"
        "\n\n1. For physical actions or real-world interactions:"
        "\n   - Describe the immediate, concrete results creatively and in detail."
        "\n   - Focus on what actually occurs in the environment or the next step in a plan."
        "\n   - If the attempt contradicts known facts or physical laws, describe a plausible outcome within the game world's rules."
        "\n\n2. For digital interactions (using devices, apps, or viewing online content):"
        "\n   - Do not describe any actual interaction with digital interfaces or content."
        "\n   - Focus on the character's preparation, anticipation, or mental state related to the planned digital action."
        "\n   - Describe the physical setting or actions related to preparing for the digital interaction."
        "\n   - Avoid mentioning opening apps, loading content, or any specific digital processes."
        "\n   - Keep digital actions as plans or intentions rather than executed actions."
        "\n   - If the attempt mentions a direct quote they are thinking of (like a post or message they are planning), retain that quote in your response."
        "\n\n3. General guidelines:"
        '\n   - Your response should naturally continue the attempted action, suitable to follow "Because of that,".'
        "\n   - Do not repeat the original action attempt in your response."
        "\n   - For simple or routine actions, provide a brief, matter-of-fact description of the outcome."
        "\n\nExamples:"
        '\n1. Attempt: "Alice is preparing to check her Mastodon feed to catch up on the latest news and updates from her community. '
        'She sits at her kitchen table, her smartphone in hand, and opens the MastodonSocialNetworkApp."'
        '\nResult: "Alice sits at her kitchen table with her smartphone, anticipating the latest updates from her community. '
        "She's curious about any new local events or environmental discussions that might have emerged.\""
        '\n2. Attempt: "John jumps off a cliff, aiming to fly like a superhero."'
        '\nResult: "John leaps from the cliff edge, but instead of flying, he plummets rapidly towards the water below. '
        'The wind rushes past him as gravity takes hold, his stomach lurching with the sudden descent."'
        '\n3. Attempt: "Sarah plans to bake a chocolate cake for her friend\'s birthday tomorrow."'
        '\nResult: "She mentally reviews the ingredients she\'ll need to buy from the store, imagining the rich aroma of cocoa that will soon fill her kitchen."'
        '\n4. Attempt: "Mark opens the heavy oak door to enter the library."'
        '\nResult: "The door creaks open, revealing rows of bookshelves. The scent of old books wafts out as Mark steps inside, '
        'dust motes dancing in the slanted afternoon sunlight streaming through tall windows."'
        '\n5. Attempt: "Emily checks the weather forecast on her phone app for the weekend."'
        '\nResult: "Emily glances out the window, wondering if the weekend will be suitable for outdoor activities. '
        'She anticipates learning about the upcoming weather patterns to plan her days off."'
        "\n6. Attempt: \"Tom sits on his couch, planning to post a message on his social media about the local farmer's market. "
        "He thinks about writing, 'Don't miss the spring farmer's market this Saturday! Fresh produce and handmade crafts galore!'\""
        "\nResult: \"Tom relaxes on his couch, contemplating the farmer's market post he wants to share. "
        "He mentally refines the message, 'Don't miss the spring farmer's market this Saturday! Fresh produce and handmade crafts galore!', "
        'imagining how it might encourage his friends to attend."'
        "\nNow, based on these examples and guidelines, provide an appropriate result for the given action attempt:",
        max_tokens=2000,
    )
    raw_causal_statement = f"{action_attempt} Because of that, {result}"
    return raw_causal_statement


def custom_result_to_who_what_where(
    chain_of_thought: interactive_document.InteractiveDocument,
    event: str,
    active_player_name: str,
):
    """Determines who have done what where, given the event.

    Args:
      chain_of_thought: the document to condition on and record the thoughts
      event: the event to determine the causal outcome of
      active_player_name: name of player whose turn it currently is

    Returns:
    """
    del active_player_name
    chain_of_thought.statement(event)
    causal_statement = chain_of_thought.open_question(
        "Rewrite the statement above as one concise sentence that highlights"
        " the main person, what they did or plan to do, where,"
        " and what happened as a result (if anything), following these guidelines:"
        "\n1. Do not add any information not present in the original statement."
        "\n2. For physical actions, describe what actually occurred."
        "\n3. For digital interactions (using devices, apps, or online content):"
        "\n   - Keep plans and preparations as they are, without describing execution."
        '\n   - Use phrases like "plans to", "prepares to", or "gets ready to" for digital actions.'
        "\n   - Focus on the person's intention or anticipation rather than the digital action itself."
        "\n   - If there's a specific message or content planned, include it as a direct quote."
        '\n4. Do not express uncertainty (e.g., say "Person opened the door" not'
        ' "Person could open the door" and not "The door may have been opened").'
        "\n5. Ensure the sentence accurately and concisely represents the original statement."
        "\n6. Use the name given in the original statement."
        "\n\nExamples:"
        '\n- Original: "Alice sits at her kitchen table with her smartphone, composing a message for her Mastodon feed. '
        "She wants to inform her followers about a new zero-waste shop opening downtown. "
        "She thinks about writing, 'Exciting news! A new zero-waste shop is opening on Main Street next week. Who's up for a visit?'\""
        "\n  Rewrite: \"Alice plans to post on Mastodon, 'Exciting news! A new zero-waste shop is opening on Main Street next week. Who's up for a visit?', while at her kitchen table.\""
        '\n- Original: "John leaps from the cliff edge, but instead of flying, he plummets rapidly towards the water below. '
        'The wind rushes past him as gravity takes hold, his stomach lurching with the sudden descent."'
        '\n  Rewrite: "John jumps off a cliff and falls rapidly towards the water below, experiencing rushing wind and a lurching stomach."'
        '\n- Original: "Sarah mentally reviews the ingredients she\'ll need to buy from the store, imagining the rich aroma of cocoa that will soon fill her kitchen."'
        '\n  Rewrite: "Sarah plans to bake a chocolate cake, mentally listing ingredients and imagining cocoa aroma."'
        '\n- Original: "Mark opens the creaky door, revealing rows of bookshelves. The scent of old books wafts out as he steps inside, '
        'dust motes dancing in the slanted afternoon sunlight streaming through tall windows."'
        '\n  Rewrite: "Mark enters a library filled with the scent of old books and sunlit dust motes."'
        '\n- Original: "Emily glances out the window, thinking about posting a weather-related question on her social media. '
        "She considers asking her followers, 'Any recommendations for indoor activities this rainy weekend?'\""
        "\n  Rewrite: \"Emily plans to post on social media, 'Any recommendations for indoor activities this rainy weekend?', after looking out the window.\""
        "\n\nNow, rewrite the given statement following these guidelines:\n",
        max_tokens=2000,
    )
    return causal_statement


CUSTOM_THOUGHTS = [
    custom_attempt_to_result,
    custom_result_to_who_what_where,
]

In [ ]:
# Create the game master object

# Experimental version (custom call to action and thought chains)
env = game_master.GameMaster(
    model=model,
    memory=game_master_memory,
    clock=clock,
    players=players,
    action_spec=action_spec,
    components=[
        scenario_knowledge,
        player_status,
        direct_effect_externality,
        relevant_events,
        time_display,
        phone_triggering,
    ],
    randomise_initiative=True,
    player_observes_event=False,
    verbose=True,
    use_default_instructions=False,
    update_thought_chain=CUSTOM_THOUGHTS,
)

## The RUN

In [ ]:
# Set the clock to the start time
clock.set(START_TIME)

In [ ]:
# Add some observations to seed the players with
for player in players:
    player.observe(f"{player.name} is at home, they have just woken up.")
    player.observe(f"{player.name} remembers they want to update their Mastodon bio.")
    player.observe(
        f"{player.name} remembers they want to read their Mastodon feed to catch up on news"
    )

In [ ]:
# Run an episode for a few steps (expect about 1 minute per step)
episode_length = 3
for _ in range(episode_length):
    env.step()

## Summary and analysis of the episode

In [ ]:
# Summarize the entire story
all_gm_memories = env._memory.retrieve_recent(k=10000, add_time=True)

detailed_story = "\n".join(all_gm_memories)
print("len(detailed_story): ", len(detailed_story))
# print(detailed_story)

episode_summary = model.sample_text(
    f"Sequence of events:\n{detailed_story}"
    "\nNarratively summarize the above temporally ordered "
    "sequence of events. Write it as a news report. Summary:\n",
    max_tokens=3500,
    terminators=(),
)
print(episode_summary)

In [ ]:
# Summarise the perspective of each player
player_logs = []
player_log_names = []
for player in players:
    name = player.name
    detailed_story = "\n".join(memories[player.name].retrieve_recent(k=1000, add_time=True))
    summary = ""
    summary = model.sample_text(
        f"Sequence of events that happened to {name}:\n{detailed_story}"
        "\nWrite a short story that summarises these events.\n",
        max_tokens=3500,
        terminators=(),
    )

    all_player_mem = memories[player.name].retrieve_recent(k=1000, add_time=True)
    all_player_mem = ["Summary:", summary, "Memories:", *all_player_mem]
    player_html = html_lib.PythonObjectToHTMLConverter(all_player_mem).convert()
    player_logs.append(player_html)
    player_log_names.append(f"{name}")

## Build and display HTML log of the experiment

In [ ]:
history_sources = [env, direct_effect_externality]
histories_html = [
    html_lib.PythonObjectToHTMLConverter(history.get_history()).convert()
    for history in history_sources
]
histories_names = [history.name for history in history_sources]

In [ ]:
gm_mem_html = html_lib.PythonObjectToHTMLConverter(all_gm_memories).convert()

tabbed_html = html_lib.combine_html_pages(
    [*histories_html, gm_mem_html, *player_logs],
    [*histories_names, "GM", *player_log_names],
    summary=episode_summary,
    title="Mastodon experiment",
)

tabbed_html = html_lib.finalise_html(tabbed_html)

In [ ]:
display.HTML(tabbed_html)

## Interact with a specific player

In [ ]:
sim_to_interact = "Alice"
user_identity = "a close friend"
interaction_premise = f"{sim_to_interact} is talking to {user_identity}\n"

player_names = [player.name for player in players]
player_by_name = {player.name: player for player in players}
selected_player = player_by_name[sim_to_interact]
interrogation = interaction_premise

In [ ]:
utterance_from_user = "Hey Alice, did you post anything on Mastodon today?"

interrogation += f"{user_identity}: {utterance_from_user}"
player_says = selected_player.say(interrogation)
interrogation += f"\n{sim_to_interact}: {player_says}\n"
print(interrogation)

## Check timeline

Finally, check the full public timeline for the Mastodon server.

You may also check this directly at https://social-sandbox.com (you'll need to log in as a user).

In [ ]:
timeline = get_public_timeline(limit=None)
print_timeline(timeline)